In [ ]:
import pandas as pd
import numpy as np
import datetime
import backtrader as bt
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from backtrader.feeds import PandasData
import backtrader.indicators as btind
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
import multiprocessing
from pyswarm import pso
from xgboost import XGBRegressor
from backtrader.analyzers import SharpeRatio, DrawDown, TradeAnalyzer
plt.rcParams["font.sans-serif"]=["SimHei"]  # 设置画图时的中文显示
plt.rcParams["axes.unicode_minus"]=False  # 设置画图时的负号显示

data = pd.read_csv(open(r'E:\Users\OneDrive\桌面\futures_data\AG9999.XSGE.csv'))
stock_df=data[['open','high','low','close','volume','open_interest','date']]
#stock_df.index=data['date']

fromdate=datetime.datetime(2012,5,1)
todate=datetime.datetime(2024,5,7)
# data=bt.feeds.PandasData(dataname=stock_df,fromdate=fromdate,todate=todate,datetime='date')  # 加载数据
data = bt.feeds.GenericCSVData(
    dataname=r'E:\Users\OneDrive\桌面\futures_data\AG9999.XSGE.csv',
    datetime=0,  # 'date'列的索引为6
    fromdate=pd.Timestamp('2012-05-01'),
    todate=pd.Timestamp('2024-05-07'),
    nullvalue=0.0,
    dtformat='%Y-%m-%d',  # 日期格式
    datetimeformat='%Y-%m-%d',  # 日期时间格式
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=6,
    timeframe=bt.TimeFrame.Days,
    compression=1
)
import warnings
warnings.filterwarnings("ignore")
stock_df

In [ ]:
# 构建策略
accuracies = [] #记录正确率
total_money = [] #记录资金
class BollingerBandsStrategy(bt.Strategy):
    params = (
        ('period', 20),
        ('devfactor', 2),
    )

    def __init__(self):
        self.bollinger = bt.indicators.BollingerBands(
            self.data.close, period=self.params.period,
            devfactor=self.params.devfactor
            self.start_cash = 62000
            self.trade_analyzer = bt.analyzers.TradeAnalyzer()
            self.entry_price = 0
            self.price_record = []
        )

    def next(self):
        # 交易时间
        current_date = self.datas[0].datetime.date(0)
        if not datetime.datetime(2024,3,1).date()<=current_date<=datetime.datetime(2024,5,8).date():
            return
        
        if self.position:  # 如果当前有持仓
            if self.data.close[0] > self.bollinger.lines.top:  # 如果价格高于上轨
                self.sell(size=1)  # 卖出
            elif self.data.close[0] < self.bollinger.lines.mid:  # 如果价格低于中轨
                self.buy(size=1)  # 买入
        else:  # 如果当前没有持仓
            if self.data.close[0] < self.bollinger.lines.bottom:  # 如果价格低于下轨
                self.buy(size=1)  # 买入